In [1]:
import EOSutils as meos

import numpy as np
import matplotlib.pyplot as plt

import matplotlib.style
import matplotlib as mpl
import matplotlib.colors as colors
import matplotlib.cm as cmx
from mpl_toolkits.axes_grid1 import make_axes_locatable

mpl.style.use('classic')

from astropy.table import Table
from astropy import units as u
from astropy.constants import G

import mesa_helper as mh
import os
import shutil
import copy

from scipy import interpolate
from scipy.linalg import lstsq as scipy_lstsq
from scipy.ndimage import gaussian_filter
import scipy.integrate as integrate
import decimal 

%matplotlib inline

# Collins+1998

In [5]:
data = Table.read("./D2_collins+1998_digitized_withunc.dat",format="ascii")
data.rename_column('col1','rho')
data.rename_column('col2','P')

In [6]:
print(data)

        rho                 P         
------------------- ------------------
0.47539631048707937 10.083143168206146
0.47539631048707937 10.083143168206146
 0.4891652691238937  10.50417116148882
  0.524733446768922 14.254903245870675
 0.5073508709649778 13.770163674238727
 0.5418809905619958 14.652722462667878
 0.5629163555348815 19.931113245514194
 0.5478253418369221 19.425600082077878
 0.5781052992373701 20.436971777000743
 0.5685865027971257 23.758754128338836
                ...                ...
 1.1791996670379845 239.31644058346603
 0.9997135547367518 264.47000669914024
 0.8351030101235142 258.65209415562236
 1.1656657404120403 270.64883720937115
 0.8987281340661761  284.4469000553744
  0.698588583809722  267.7691919424497
 1.1009438004186507  306.6593214911254
 0.8907566316974942   322.206713476607
 0.7730643522542262 311.97526515025913
 1.0073423020895813 330.57671582322865
Length = 66 rows


In [9]:
med = data[0::3]
lower = data[1::3]
upper = data[2::3]
print(len(med),len(lower),len(upper))


22 22 22


In [15]:
lower_unc_rho = med['rho'] - lower['rho']
upper_unc_rho = upper['rho'] - med['rho']

lower_unc_P = med['P'] - lower['P']
upper_unc_P = upper['P'] - med['P']

new_data = Table()

new_data.add_column(np.round(med['rho'],5),name='rho[g/cm3]')
new_data.add_column(np.round(lower_unc_rho,5), name='rho_unc_lower[g/cm3]')
new_data.add_column(np.round(upper_unc_rho,5), name='rho_unc_upper[g/cm3]')
new_data.add_column(np.round(med['P'],5),name='P[GPa]')
new_data.add_column(np.round(lower_unc_P,5), name='P_unc_lower[GPa]')
new_data.add_column(np.round(upper_unc_P,5), name='P_unc_upper[GPa]')

In [16]:
print(new_data)

rho[g/cm3] rho_unc_lower[g/cm3] ... P_unc_lower[GPa] P_unc_upper[GPa]
---------- -------------------- ... ---------------- ----------------
    0.4754                  0.0 ...              0.0          0.42103
   0.52473              0.01738 ...          0.48474          0.39782
   0.56292              0.01509 ...          0.50551          0.50586
   0.56859              0.02342 ...          1.85859           1.9564
   0.59414              0.05838 ...          1.70847          1.63163
   0.79817              0.15494 ...          2.37123          1.67087
   0.93913              0.06144 ...          1.09323          1.47146
   0.94234              0.16419 ...          3.43455          2.48398
   0.90124              0.06456 ...          5.13835          5.00613
   0.99273              0.11137 ...          4.75709          4.90346
   1.00136              0.04093 ...          5.12491          3.85194
   1.03048              0.15822 ...          4.50316          2.69201
   0.99867          

In [17]:
new_data.write("./D2_collins+1998.dat", format='ascii')

# Knudson+2009

## their corrected version of Hicks+2009

In [5]:
data = Table.read("./D2_hicks+2009_via_knudson+2009.dat",format="ascii")
data.rename_column('col1','eta') #compression ratio, rho/rho_0
data.rename_column('col2','P')
print(data)

med = data[0::3]
lower = data[1::3]
upper = data[2::3]
print(len(med),len(lower),len(upper))

lower_unc_eta = med['eta'] - lower['eta']
upper_unc_eta = upper['eta'] - med['eta']

lower_unc_P = med['P'] - lower['P']
upper_unc_P = upper['P'] - med['P']

new_data = Table()

new_data.add_column(np.round(med['eta'],5),name='rho/rho_0[unitless')
new_data.add_column(np.round(lower_unc_eta,5), name='rho/rho_0_unc_lower[unitless]')
new_data.add_column(np.round(upper_unc_eta,5), name='rho/rho_0_unc_upper[unitless]')
new_data.add_column(np.round(med['P'],5),name='P[GPa]')
new_data.add_column(np.round(lower_unc_P,5), name='P_unc_lower[GPa]')
new_data.add_column(np.round(upper_unc_P,5), name='P_unc_upper[GPa]')

       eta                 P         
------------------ ------------------
3.3985459848379245 44.364478029213345
 3.002380467300743 44.359919774720936
 3.796791730556836  44.29154595733474
 3.950921626867112  73.92171957618362
3.5335892514395395  74.02655942950912
4.3708703717603665  73.77737485059055
3.6488596274890357  83.89821924191163
3.3073590752635136  83.94228236867163
 3.993877266537278  83.67334535361925
3.7320473091068957   97.4818176293025
               ...                ...
 4.909370677360898 180.54992808087357
 4.359804415659615 182.35803569619742
 3.945839007495255 179.07153420716762
4.7701669543958225 187.56963999918966
 4.360898143878875 202.05121452158588
 3.983626245188132 196.00848848281032
 4.741794357649128  207.5196004943174
  4.52699471364694  218.4563724397804
 4.115881577113201 213.11865642916473
 4.934762328568825 223.35649601912442
Length = 45 rows
15 15 15


In [6]:
print(new_data)

rho/rho_0[unitless rho/rho_0_unc_lower[unitless] ... P_unc_upper[GPa]
------------------ ----------------------------- ... ----------------
           3.39855                       0.39617 ...         -0.07293
           3.95092                       0.41733 ...         -0.14434
           3.64886                        0.3415 ...         -0.22487
           3.73205                       0.33852 ...         -0.07293
           4.32897                       0.46906 ...         -0.08509
           4.36802                       0.43533 ...         -0.05774
           4.26396                        0.4281 ...         -0.23703
           4.03458                       0.34725 ...          -0.3206
           4.13778                       0.36057 ...          0.09572
           4.47829                       0.42525 ...          -0.1565
           4.66062                        0.4655 ...          3.64508
           4.48346                       0.42362 ...          2.68481
            4.3598  

In [7]:
new_data.write("./D2_hicks+2009.dat", format='ascii')

## their Knudson+2004 with additional unpublished data

In [8]:
data = Table.read("./D2_knudson+2004_via_knudson+2009_digitized.dat",format="ascii")
data.rename_column('col1','eta') #compression ratio, rho/rho_0
data.rename_column('col2','P')
print(data)

med = data[0::3]
lower = data[1::3]
upper = data[2::3]
print(len(med),len(lower),len(upper))

lower_unc_eta = med['eta'] - lower['eta']
upper_unc_eta = upper['eta'] - med['eta']

lower_unc_P = med['P'] - lower['P']
upper_unc_P = upper['P'] - med['P']

new_data = Table()

new_data.add_column(np.round(med['eta'],5),name='rho/rho_0[unitless')
new_data.add_column(np.round(lower_unc_eta,5), name='rho/rho_0_unc_lower[unitless]')
new_data.add_column(np.round(upper_unc_eta,5), name='rho/rho_0_unc_upper[unitless]')
new_data.add_column(np.round(med['P'],5),name='P[GPa]')
new_data.add_column(np.round(lower_unc_P,5), name='P_unc_lower[GPa]')
new_data.add_column(np.round(upper_unc_P,5), name='P_unc_upper[GPa]')

       eta                 P         
------------------ ------------------
 3.533298268954102 22.031970130097427
 3.254474575746794   21.6053614141532
 3.816051235245889  22.06478618517006
3.4711415002397783  22.36450615483344
3.1900592484878487 22.373257102852808
 3.752595021889454  22.47608074208039
 4.320761722073542 30.874803103669564
3.6346240118806366 30.922933317776092
 5.019306034682796  30.36287264453649
 4.315873335086552 31.305787293623474
               ...                ...
 4.484955834351747 118.78026369523366
 4.230450319446808 120.57639577620908
 3.914561514781183 120.53920424712678
4.5545070618628465 120.82142232075141
  4.01022539176709  127.8987515314159
  3.68363163838312 124.05270987690334
 4.341521897189177 131.60477801761857
 4.311944062002073 175.50172101977714
 3.958897328403694   170.485240067674
4.6635057160094675 179.01741438655856
Length = 96 rows
32 32 32


In [9]:
print(new_data)

rho/rho_0[unitless rho/rho_0_unc_lower[unitless] ... P_unc_upper[GPa]
------------------ ----------------------------- ... ----------------
            3.5333                       0.27882 ...          0.03282
           3.47114                       0.28108 ...          0.11157
           4.32076                       0.68614 ...         -0.51193
           4.31587                         0.563 ...         -0.08095
           3.86948                       0.39673 ...          0.18158
           3.86948                       0.23486 ...          0.37848
           4.55336                       0.80049 ...          0.19252
           4.06112                       0.36944 ...          1.34546
           4.14469                       0.40221 ...          -0.1422
           4.21408                       0.34095 ...          1.92083
               ...                           ... ...              ...
           3.99735                       0.29581 ...          0.75477
           4.32701  

In [10]:
new_data.write("./D2_knudson+2004.dat", format='ascii')

# Eggert + 2008

In [11]:
data = Table.read("./He_eggert+2008_digitized.dat",format="ascii")
data.rename_column('col1','eta') #compression ratio, rho/rho_0
data.rename_column('col2','P')
print(data)

med = data[0::3]
lower = data[1::3]
upper = data[2::3]
print(len(med),len(lower),len(upper))

lower_unc_eta = med['eta'] - lower['eta']
upper_unc_eta = upper['eta'] - med['eta']

lower_unc_P = med['P'] - lower['P']
upper_unc_P = upper['P'] - med['P']

new_data = Table()

new_data.add_column(np.round(med['eta'],5),name='rho/rho_0[unitless')
new_data.add_column(np.round(lower_unc_eta,5), name='rho/rho_0_unc_lower[unitless]')
new_data.add_column(np.round(upper_unc_eta,5), name='rho/rho_0_unc_upper[unitless]')
new_data.add_column(np.round(med['P'],5),name='P[GPa]')
new_data.add_column(np.round(lower_unc_P,5), name='P_unc_lower[GPa]')
new_data.add_column(np.round(upper_unc_P,5), name='P_unc_upper[GPa]')

       eta                P         
----------------- ------------------
5.381277756073447 51.015322661259226
  4.7786370920804  42.95414625787196
6.130462514849748  58.13828951690063
6.342305637721949  85.89329829922744
5.581360675474196  75.33879789224156
7.385017102126404  94.39287703314436
6.113759039410635 117.33688434460994
5.283847460194702  99.95930141231239
7.096513398658938 131.63993888071744
3 3 3


In [12]:
print(new_data)

rho/rho_0[unitless rho/rho_0_unc_lower[unitless] ... P_unc_upper[GPa]
------------------ ----------------------------- ... ----------------
           5.38128                       0.60264 ...          7.12297
           6.34231                       0.76094 ...          8.49958
           6.11376                       0.82991 ...         14.30305


In [15]:
new_data.write("./He_eggert+2008.dat", format='ascii',overwrite=True)